# Optimizing the model with series of experiments

In [1]:
import pathlib

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

### Let´s first reload the data from the previous notebook. Then, perform train_test_split once again.

In [2]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

/home/arthur/Documentos/Insper/6_semestre/projetos_ml/AmesHousingDataset/data


In [3]:
model_data_scaled_path = DATA_DIR / 'processed' / 'ames_model_data_scaled.pkl'

In [4]:
data = pd.read_pickle(model_data_scaled_path)

In [5]:
X = data.drop(columns=['SalePrice']).copy().values
y = data['SalePrice'].copy().values

In [6]:
X.shape, y.shape

((2877, 164), (2877,))

In [8]:
RANDOM_SEED = 42  # Any number here, really.

Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)

### Experiment 1: LinearRegression with encoded categorical values AND scaled continuous variables

Our first experiment is to rerun the linear regression model with the same features as before, but this time on the new data with scaled continuous variables. 

In [9]:
from sklearn.linear_model import LinearRegression

In [10]:
linear_scaled_model = LinearRegression()

linear_scaled_model.fit(Xtrain, ytrain)

LinearRegression()

In [24]:
scores_linear_scaled = cross_val_score(linear_scaled_model,
                                       Xtrain, 
                                       ytrain,
                                       cv=5,
                                       scoring='neg_mean_squared_error',
                                       n_jobs=-1)

scores_linear_scaled = np.sqrt(-scores_linear_scaled)
print(f'MEAN: {scores_linear_scaled.mean():.2f} +/- {scores_linear_scaled.std():.2f}')
error_percent_linear = 100 * (10**scores_linear_scaled.mean() - 1)
print(f'Average error is {error_percent_linear:.2f}%')

MEAN: 0.06 +/- 0.01
Average error is 14.02%


The experiment showed no signifficant difference in the model performance. This means that the LinearRegression model is not sensitive to the scale of the continuous variables. We will, however, still keep the scaled data for the next experiments.

### Experiment 2: Lasso Regression

In [28]:
from sklearn.linear_model import Lasso

In [29]:
lasso = Lasso()

params = {
    'alpha': np.logspace(-4, 0, 100)
}

grid = GridSearchCV(lasso, params, cv=5, verbose=1, n_jobs=-1)

In [30]:
grid.fit(Xtrain, ytrain)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/arthur/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.580e-02, tolerance: 4.972e-03
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=5, estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-04, 1.09749877e-04, 1.20450354e-04, 1.32194115e-04,
       1.45082878e-04, 1.59228279e-04, 1.74752840e-04, 1.91791026e-04,
       2.10490414e-04, 2.31012970e-04, 2.53536449e-04, 2.78255940e-04,
       3.05385551e-04, 3.35160265e-04, 3.67837977e-04, 4.03701726e-04,
       4.43062146e-04, 4.86260158e-04, 5.33669...
       1.17681195e-01, 1.29154967e-01, 1.41747416e-01, 1.55567614e-01,
       1.70735265e-01, 1.87381742e-01, 2.05651231e-01, 2.25701972e-01,
       2.47707636e-01, 2.71858824e-01, 2.98364724e-01, 3.27454916e-01,
       3.59381366e-01, 3.94420606e-01, 4.32876128e-01, 4.75081016e-01,
       5.21400829e-01, 5.72236766e-01, 6.28029144e-01, 6.89261210e-01,
       7.56463328e-01, 8.30217568e-01, 9.11162756e-01, 1.00000000e+00])},
             verbose=1)

In [31]:
best_params = grid.best_params_
print(best_params)

{'alpha': 0.0001}


In [32]:
lasso_best = Lasso(**best_params)
lasso_best.fit(Xtrain, ytrain)

Lasso(alpha=0.0001)

In [33]:
scores_lasso = cross_val_score(lasso_best, Xtrain, ytrain, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
scores_lasso = np.sqrt(-scores_lasso)
print(f'MEAN: {scores_lasso.mean():.2f} +/- {scores_lasso.std():.2f}')
error_percent_lasso = 100 * (10**scores_lasso.mean() - 1)
print(f'Average error is {error_percent_lasso:.2f}%')

MEAN: 0.06 +/- 0.01
Average error is 13.80%


/home/arthur/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.580e-02, tolerance: 4.972e-03
  model = cd_fast.enet_coordinate_descent(


### Experiment 3:  Random Forest Regression

In [35]:
from sklearn.ensemble import RandomForestRegressor

In [36]:
rf = RandomForestRegressor()

params = {
    'n_estimators': [50, 100, 200, 300, 400, 500, 600],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.2],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 10, 20, 50, 100],
    'min_impurity_decrease': [0.0, 0.1, 0.2],
    'bootstrap': [True, False],
    'oob_score': [True, False],
    'verbose': [0, 1, 2],
    'warm_start': [True, False],
    'ccp_alpha': [0.0, 0.1, 0.2],
}

grid = GridSearchCV(rf, params, cv=5, verbose=1, n_jobs=-1)

In [37]:
grid.fit(Xtrain, ytrain)

Fitting 5 folds for each of 1088640 candidates, totalling 5443200 fits


In [ ]:
best_params = grid.best_params_
print(best_params)

In [ ]:
rf_best = RandomForestRegressor(**best_params)
rf_best.fit(Xtrain, ytrain)

In [ ]:
ypred_rf = rf_best.predict(Xtest)

In [ ]:
RMSE = np.sqrt(mean_squared_error(ytest, ypred_rf))
error_percent = 100 * (10**RMSE - 1)
print(f'RMSE: {RMSE}')
print(f'Average error is {error_percent:.2f}%')

### Experiment 4:  Gradient Boosting Regression

### Experiment 5:  KNN Regression

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor()

params = {
    'n_neighbors': [5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'p': [1, 2, 3],
}

grid = GridSearchCV(knn, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
best_params = grid.best_params_
print(best_params)

In [ ]:
knn_best = KNeighborsRegressor(**best_params)
knn_best.fit(Xtrain, ytrain)

In [ ]:
y_pred_knn = knn_best.predict(Xtest)

In [ ]:
RMSE = np.sqrt(mean_squared_error(ytest, y_pred_knn))
error_percent = 100 * (10**RMSE - 1)
print(f'RMSE: {RMSE}')
print(f'Average error is {error_percent:.2f}%')

### Experiment 6:  Regression Stack

### Retraining of best model on the whole dataset

In [ ]:
best_model = linear_scaled_model

In [ ]:
best_model.fit(X, y)